<a href="https://colab.research.google.com/github/aman4312/dukaan_bot/blob/main/updated_dukaan_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot==13.13

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.0
    Uninstalling cachetools-5.3.0:
      Successfully uninstalled cachetools-5.3.0


In [ ]:
import telegram
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, ReplyKeyboardMarkup, Update
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler,InlineQueryHandler , MessageHandler, Filters
from collections import defaultdict

TOKEN = "6037452027:AAEY1RfgLzUr8AsUmYKO81PK0bgU3strOKI"
OWNER_ID = "1366093138" 
ITEMS = [
        {'name': 'Mustard Oil', 'price': 100.0, 'quantity': 1, 'type': 'Oil', 'brand': 'Fortune'},
        {'name': 'Olive Oil', 'price': 200.0, 'quantity': 1, 'type': 'Oil', 'brand': 'Emami'},
        {'name': 'Palm Oil', 'price': 150.0, 'quantity': 1, 'type': 'Oil', 'brand': 'Fortune'},
        {'name': 'Sugar', 'price': 40.0, 'quantity': 1, 'type': 'Food', 'brand': 'Madhur'},
        {'name': 'Tea', 'price': 250.0, 'quantity': 1, 'type': 'Food', 'brand': 'Tata'},
        {'name': 'Pickle', 'price': 70.0, 'quantity': 1, 'type': 'Food', 'brand': 'Mathur'},
        {'name': 'Tur dal', 'price': 80.0, 'quantity': 1, 'type': 'Food', 'brand': 'Tata Sampann'},
        {'name': 'Rice Flour', 'price': 60.0, 'quantity': 1, 'type': 'Food', 'brand': 'Tata Sampann'},
        {'name': 'Kala Chana', 'price': 90.0, 'quantity': 1, 'type': 'Food', 'brand': 'Tata Sampann'},
        {'name': 'Rice Bran Oil', 'price': 100.0, 'quantity': 1, 'type': 'Food', 'brand': 'Fortune'},
        {'name': 'Salt', 'price': 30.0, 'quantity': 1, 'type': 'Food', 'brand': 'Tata'},
        {'name': 'Methi', 'price': 100.0, 'quantity': 1, 'type': 'Food', 'brand': 'Tata Sampann'},
    ]
ITEMS_KEYBOARD = [[InlineKeyboardButton(item["name"], callback_data=item["name"])] for item in ITEMS]
ITEMS_KEYBOARD.append([InlineKeyboardButton("Confirm", callback_data="confirm"), InlineKeyboardButton("Cancel", callback_data="cancel")])
DELIVERY_KEYBOARD = [[InlineKeyboardButton("Delivery", callback_data="delivery"), InlineKeyboardButton("Pick-up", callback_data="pickup")]]
temp_id = None
POINTS = defaultdict(lambda: defaultdict(int))

def format_item(item):
    return f"| {item['name']}| Rs{item['price']}| {item['quantity']}| {item['quantity']} kg| {item['type']}| {item['brand']}|\n\n"

def format_order(order):
    return "\n".join([f"- {item['name']} ({item['quantity']} kg)" for item in order])

def format_cost(order):
    return sum([item["price"] for item in order])

def send_message_to_owner(context, user_id, order):
    context.bot.send_message(
        chat_id=OWNER_ID,
        text=f"A new order has been placed by @{user_id}. The order details are:\n\n{format_order(order)}\n\nTotal cost: Rs{format_cost(order)}\n\nPlease accept or decline the order by pressing [ Accept ] or [ Decline ].",
        reply_markup=InlineKeyboardMarkup([[InlineKeyboardButton("Accept", callback_data="accept"), InlineKeyboardButton("Decline", callback_data="decline")]])
    )

def send_message_to_user(context, user_id, text):
    context.bot.send_message(
        chat_id=user_id,
        text=text,
        parse_mode=telegram.ParseMode.MARKDOWN
    )

def start(update, context):
    user_id = update.message.from_user.id
    send_message_to_user(context, user_id, f"Hello, this is a telegram bot for a local general store. Please select the items you want to order:")

    table_header = "| Item| Price| Quantity| Weight| Type| Brand|\n| ---------| ---------| ---------| ---------| ---------| ---------|\n"
    table_body = "".join([format_item(item) for item in ITEMS])
    table = table_header + table_body

    send_message_to_user(context, user_id, table + 'For more information about these products do visit our channel: ' + 'https://t.me/latest%5Fdeals%5Fdukaan')

    context.bot.send_message(
        chat_id=user_id,
        text="Please select the items you want to order:",
        reply_markup=InlineKeyboardMarkup(ITEMS_KEYBOARD)
    )

def pay(update, context):
    user_id = update.message.from_user.id
    send_message_to_user(context, user_id, f"This feature will be implemented soon")
    
def share(update, context):
    user_id = update.message.from_user.id
    
    share_message = 'Your share means a lot to us! Please let your friends know they can get amazing deals and products from our e-commerce store.'
    share_button = InlineKeyboardButton('Share', switch_inline_query="Hey there! your friend shared our e-commerce bot with you, and we're thrilled to have you check us out. Explore our products and find great deals today.")
    share_markup = InlineKeyboardMarkup([[share_button]])
    context.bot.send_message(chat_id=user_id, text=share_message, reply_markup=share_markup)
    
    context.user_data['share'] = 1
    send_message_to_user(context, user_id, f"To confirm your share please type 'shared' below and earn your 10 points.")
    
    
def shared(update, context):
    user_id = update.message.from_user.id
    msg = update.message.text
    if (msg == "shared" or msg == "Shared") and context.user_data.get('share', 0):
        if POINTS[user_id]['shared'] < 10:
            POINTS[user_id]['shared'] += 1
            POINTS[user_id]['points'] += 10
            send_message_to_user(context, user_id, f"Thank you for sharing our bot! We aprreciate your support and hope your friends enjoy it. You have succesfully earned 10 points.")
        else:
            send_message_to_user(context, user_id, f"Sorry!, You have exceeded your limit of earning from refer and earn.")
        
        context.user_data['share'] = 0
    
    else:
        send_message_to_user(context, user_id, f"Sorry! To earn more points you have to share.")

def feedback(update, context):
    user_id = update.message.from_user.id
    send_message_to_user(context, user_id, f"Please, write Your Feedback.\nYour Feedback should start with 'Feedback '")
    
def r_feedback(update, context):
    user_id = update.message.from_user.id
    
    msg = update.message.text.replace('feedback', '').replace('Feedback', '')
    if msg and not POINTS[user_id]['feedback']:
        send_message_to_user(context, user_id, f"You have successfully earned 10 points!.\nThank you for providing your valuable feedback!.")
        POINTS[user_id]['points'] += 10
        POINTS[user_id]['feedback'] = msg
        print(POINTS[user_id]['feedback'])
    elif POINTS[user_id]['feedback']:
        send_message_to_user(context, user_id, f"Sorry!, You can't write the feedback again.")
    else:
        send_message_to_user(context, user_id, f"Please, write something.")
    
def points(update, context):
    send_message_to_user(context, update.message.from_user.id, f"You have total {POINTS[update.message.from_user.id]['points']} points")

def redeem(update, context):
    user_id = update.message.from_user.id
    send_message_to_user(context, user_id, f"This feature will be implemented soon")
    
def help(update, context):
    user_id = update.message.from_user.id
    send_message_to_user(context, user_id, f"This bot can help you order items from a local general store.\nTo start a new order, type /start. To cancel an order, type /cancel.\n" + 
                                            "You can pay online. To pay Type /pay (This option will be implemented in future.)\n"+
                                            "You can check your total points by typing /points.\n\nYou can earn some points by doing shopping with us or by doing few tasks like.\n" +
                                            "1. Share this app with your friends and family to earn 10 points(Upto 10 friends). To share type /share\n" +
                                            "2. Write your valuable feedback to earn 10 points. To write your feedback type /feedback\n"+
                                            "You can redeem your points. To redeem Type /redeem (This option will be implemented in future.)")

def cancel(update, context):
    user_id = update.message.from_user.id
    send_message_to_user(context, user_id, f"Your order has been cancelled. To start a new order, type /start.")

def handle_items(update, context):
    global temp_id
    query = update.callback_query
    user_id = query.from_user.id

    temp_id = user_id
    item_name = query.data

    if item_name == "confirm":
        order = context.user_data.get("order", [])
        if not order:
            send_message_to_user(context, user_id, f"You have not selected any items. Please select at least one item to proceed.")
            return

        send_message_to_user(context, user_id,
                             f"You have confirmed your order. Your total cost is Rs{format_cost(order)}." +
                             f"The items included in your order are:\n\n{format_order(order)}" +
                             "\n\nDo you want to add more quantity?")

        context.bot.send_message(
         chat_id=user_id ,
         text="Please press [ Yes ] or [ No ].",
         reply_markup=InlineKeyboardMarkup([[InlineKeyboardButton("Yes", callback_data="Yes"), InlineKeyboardButton("No", callback_data="No")]])
         )
    
    elif item_name == "Yes" and context.user_data.get("order", []):
        send_message_to_user(context, user_id, "Please Enter the New quantity seperated by ',' according to your orders.\nExample: qauntity of item1,quantity of item2.")
        return
    
    elif item_name == "No" and context.user_data.get("order", []):
        send_message_to_owner(context ,user_id , context.user_data["order"])
        context.bot.send_message(
            chat_id=user_id,
            text="Please choose your preferred option for delivery or pick-up:",
            reply_markup=InlineKeyboardMarkup(DELIVERY_KEYBOARD)
        )
        
    elif item_name == "delivery" or item_name == "pickup":
        handle_delivery(update, context)

    elif item_name == "cancel":
        context.user_data.clear()
        send_message_to_user(context,user_id ,f"Your order has been cancelled. To start a new order, type /start.")
        
    else:
        order = context.user_data.get("order", [])
        
        item = next((i for i in ITEMS if i["name"] == item_name), None)
        
        if not item:
            send_message_to_user(context,user_id ,f"Sorry, I don't recognize that item. Please select one of the available items.")
            return
        
        if item in order:
            order.remove(item)
            send_message_to_user(context,user_id ,f"You have removed {item_name} from your order.")
            
        else:
            order.append(item)
            send_message_to_user(context,user_id ,f"You have added {item_name} to your order.")
            
        context.user_data["order"] = order

def change_qty(update, context):
    user_id = update.message.from_user.id 
    
    qty = update.message.text
    if not context.user_data.get('order', []):
        return
    
    if ',' in qty:
        qty = qty.split(',')
        order = context.user_data.get('order', [])
        for i in range(len(qty)):
            order[i]['price'] *= int(qty[i])
            order[i]['quantity'] = int(qty[i])
            
        send_message_to_owner(context ,user_id , order)
        send_message_to_user(context, user_id,
                             f"You have confirmed your order and quantity. Your total cost is Rs{format_cost(order)}." +
                             f"The items included in your order are:\n\n{format_order(order)}")
        context.bot.send_message(
            chat_id=user_id,
            text="Please choose your preferred option for delivery or pick-up:",
            reply_markup=InlineKeyboardMarkup(DELIVERY_KEYBOARD)
        )
        
    elif qty == "delivery" or qty == "pickup":
        handle_delivery(update, context)
    
    else:
        send_message_to_user(context, user_id,
                             f"Please Enter the quantity in correct format.")
        
        
def handle_order(update ,context):
    query = update.callback_query 
    user_id = query.from_user.id 

    if str(user_id) != OWNER_ID:
        send_message_to_user(context,user_id ,f"Sorry ,you are not authorized to handle orders .Please contact @{OWNER_ID} for assistance .")
        return 

    action = query.data 

    if action == 'accept':
        send_message_to_user(context,user_id ,f"You have accepted the order .Please prepare the items for delivery or pick-up .")
        send_message_to_user(context , temp_id ,f"Your order has been accepted by the store owner .Please wait for further instructions .")

    elif action == 'decline':
        send_message_to_user(context, user_id,f"You have declined the order .Please contact @{temp_id} for more details .")
        send_message_to_user(context , temp_id,f"Your order has been declined by the store owner .Please contact @{OWNER_ID} for more details .")

    else:
        send_message_to_user(context,user_id ,f"Sorry ,I don't recognize that action .Please press [ Accept ] or [ Decline ] to handle orders .")
         
def handle_delivery(update ,context):
    query = update.callback_query 
    user_id = query.from_user.id 
     
    if str(user_id) == OWNER_ID:
        send_message_to_user(context,user_id ,f"Sorry ,you are not authorized to choose delivery or pick-up option .Please contact @{temp_id} for assistance .")
        return
    
    option = query.data
    
    if option == "delivery":
        send_message_to_user(context,user_id ,f"You have chosen delivery .Please enter your address and phone number in the format: \n\n123 Main Street ,New York ,NY 10001\n+1 234 567 8900 :")
        
        context.user_data["delivery"] = True
        
    elif option == "pickup":
        send_message_to_user(context,user_id ,f"You have chosen pick-up .Please collect your order from the store at your convenience .\n\nYou have earned 10 points!\nThank you for shopping with us ! 😊")
        POINTS[user_id]['points'] += 10
        
        context.user_data.clear()
        
    else:
        send_message_to_user(context,user_id ,f"Sorry ,I don't recognize that option .Please choose [ Delivery ] or [ Pick-up ] to proceed .")
        
def handle_address(update ,context):
    user_id = update.message.from_user.id 
     
    if not context.user_data.get("delivery" ,False):
        send_message_to_user(context,user_id ,f"Sorry ,you have not chosen delivery option .Please type /start to start a new order or /help for more information .")
        return 
     
    address = update.message.text 
    
    if (address == "Yes" or address == "No") and context.user_data.get("address" ,False):
        handle_confirmation(update, context) 
        return

    elif not address or len(address.split("\n")) != 2 or not address.split("\n")[1].startswith("+"):
        send_message_to_user(context,user_id ,f"Sorry ,that is not a valid address and phone number format .Please enter your address and phone number in this format :\n\n123 Main Street ,New York ,NY 10001\n+1 234 567 8900")
        return 
     
    send_message_to_user(context,user_id ,f"Your address and phone number are :\n\n{address}\n\nIs this correct ? Please press [ Yes ] or [ No ].")
    context.user_data["address"] = True
     
    context.bot.send_message(
         chat_id=user_id ,
         text="Please press [ Yes ] or [ No ].",
         reply_markup=ReplyKeyboardMarkup([["Yes" ,"No"]])
     )

def handle_confirmation(update ,context):
    user_id = update.message.from_user.id 
     
    if not context.user_data.get("delivery" ,False):
        send_message_to_user(context,user_id ,f"Sorry ,you have not chosen delivery option .Please type /start to start a new order or /help for more information .")
        return 
     
    confirmation = update.message.text 
     
    if confirmation == "Yes":
        send_message_to_user(context,user_id ,f"Your order will be delivered within 30 minutes by our delivery partner @deliverybot .\n\nYou have earned 10 points!\nThank you for shopping with us ! 😊")
        
        POINTS[user_id]['points'] += 10
        context.user_data.clear()
        context.bot.send_message(
             chat_id=user_id ,
             text="To start a new order ,type /start .To get more information ,type /help ." ,
            reply_markup=ReplyKeyboardMarkup([[]] ,remove_keyboard=True)
         )
         
    elif confirmation == "No":
        context.user_data["addres"] = False
        send_message_to_user(context,user_id ,f"Please re-enter your address and phone number in this format :\n\n123 Main Street ,New York ,NY 10001\n+1 234 567 8900")
         
    else:
        send_message_to_user(context,user_id ,f"Sorry ,I don't recognize that confirmation .Please press [ Yes ] or [ No ].")

updater = Updater(TOKEN)

dispatcher = updater.dispatcher

dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(CommandHandler("help", help))
dispatcher.add_handler(CommandHandler("cancel", cancel))
dispatcher.add_handler(CommandHandler("share", share))
dispatcher.add_handler(CommandHandler("feedback", feedback))
dispatcher.add_handler(CommandHandler("points", points))
dispatcher.add_handler(CommandHandler("pay", pay))
dispatcher.add_handler(CommandHandler("redeem", redeem))

dispatcher.add_handler(CallbackQueryHandler(handle_items, pattern="^(?!accept|decline).*$"))
dispatcher.add_handler(CallbackQueryHandler(handle_order, pattern="^(accept|decline)$"))


dispatcher.add_handler(MessageHandler(Filters.regex(r'^[\d,]+$') & ~Filters.command, change_qty))
dispatcher.add_handler(MessageHandler(Filters.regex(r'^(?i)feedback\S*') & ~Filters.command, r_feedback))
dispatcher.add_handler(MessageHandler(Filters.regex(r'^(?i)shared\S*') & ~Filters.command, shared))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_address))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_confirmation))

updater.start_polling()

updater.idle()



 Awesome App
